In [ ]:
# using Pkg

# ENV["PYTHON"] = "C:\\Users\\lap2r\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"
# Pkg.build("PyCall")

# Pkg.add("PyCall")
# Pkg.add("DataFrames")
# Pkg.add("DotEnv")
# Pkg.add("CSV")
# Pkg.add("Plots")
# Pkg.add("Dates")
# Pkg.add("Gadfly")
# Pkg.add("StatsPlots")
# Pkg.add("StatsBase")
# Pkg.add("JSON")
# Pkg.add("Makie")

# using Conda
# Conda.add("nltk")
# Conda.add("num2words")

#### Environment configuration

In [ ]:
using DotEnv

DotEnv.config("../.path_env", true)
alarms_data_file_path = "../" * ENV["ALARMS_DATA_FILE"]
regions_data_file_path = "../" * ENV["REGIONS_DATA_FILE"]
weather_data_file_path = "../" * ENV["WEATHER_DATA_FILE"]
tfidf_csv_path = "../" * ENV["ISW_TF_IDF_RESULT"]

### Alarms

In [ ]:
using DataFrames
using CSV

alarms = DataFrame(CSV.File(alarms_data_file_path))

println(size(alarms))

eltype.(eachcol(alarms))


plot!(size=(800,600))

In [ ]:
# first 10 rows of the alarms
first(alarms, 10)

In [ ]:
# describe dataset
describe(alarms)

In [ ]:
# first event date
println("first event date: ", minimum(alarms[!, :start]))

# max event date
println("max event date: ", maximum(alarms[!, :end]))

In [ ]:
using Dates
using Statistics

transform!(alarms, [:start, :end] => ((x, y) -> DateTime.(y, "yyyy-mm-dd HH:MM:SS") - DateTime.(x, "yyyy-mm-dd HH:MM:SS")) => :duration)

transform!(alarms, :duration => (x -> Minute.(round.(Int, Dates.value.(x) / (1000 * 60)))) => :duration)

println("minimum duration: ", minimum(alarms[!, :duration]))
println("maximum duration: ", maximum(alarms[!, :duration]))

sort!(alarms, :duration, rev=false)
println(first(alarms, 10))
sort!(alarms, :duration, rev=true)
println(first(alarms, 10))

In [ ]:
# visualize duration and corresponding counts using StatsPlots
using StatsPlots
using StatsBase
histogram(alarms[!, :duration], bins=100, xlabel="duration (minutes)", ylabel="counts", title="duration histogram")

# create a vector of durations
durations = alarms.duration
# count the frequency of each duration
duration_counts = countmap(durations)
# create a bar plot of the duration counts
bar(duration_counts, xlabel="Duration (minutes)", ylabel="Count", title="Duration Counts", legend=true, label="count", color="white", background_color="black", grid=true)

bar!(size=(800,600))


# working:
# using Plots
# plot(alarms[!,:duration], seriestype=:histogram, bins=100, xlabel="duration (minutes)", ylabel="counts", title="duration histogram")

In [ ]:
duration_counts = countmap(alarms.duration)

# duration_counts_df = DataFrame(duration_counts, [:duration, :count])

**TF-IDF**

In [ ]:
tfidf = DataFrame(CSV.File(tfidf_csv_path))

println(size(tfidf))
println(names(tfidf))
println(describe(tfidf))

using JSON
tfidf_keywords = tfidf[!, :Keywords]
tfidf_keywords = [JSON.parse(replace(tfidf_keywords[i], "'" => "\""), dicttype=Dict{String,Float64}) for i in 1:length(tfidf_keywords)]

# append tfidf with :Count column which will contain number of items in :Keywords
tfidf[!, :Count] = [length(tfidf_keywords[i]) for i in 1:length(tfidf_keywords)]

# tfidf_keywords_count = Dict{String,Int64}()
# for i in 1:length(tfidf_keywords)
#     for (k, v) in tfidf_keywords[i]
#         if haskey(tfidf_keywords_count, k)
#             tfidf_keywords_count[k] += 1
#         else
#             tfidf_keywords_count[k] = 1
#         end
#     end
# end
# tfidf_keywords_count


# show :Name to count of :Keywords like "assessment-2022-02-24" - 4343 where 4343 is a count of items in :Keywords

sort!(tfidf, [:Count], rev=true)
# show :Name :Date and :Count columns
tfidf[!, [:Name, :Date, :Count]]

# visualize :Date and :Count using StatsPlots with x - :Date and y - :Count
using Plots
gr()
# set limit to historgram

start_date = Date.("2022-02-01", "yyyy-mm-dd")
end_date = Date.(Dates.now())
p = plot(tfidf[!, :Date], tfidf[!, :Count], seriestype=:scatter, xlabel="Date", ylabel="Count",
        xlims=Dates.value.([start_date, end_date]), title="Date vs Unique words count",
        legend=true, label="count", color="white", background_color="black",
        grid=false);

plot!(size=(800, 600))
# savefig(p, "plot.png")
p

In [ ]:
wordFrequencyToDateFrame = DataFrame()
# word_collection = Dict{String,Int64}()

# for (root, dirs, files) in walkdir("../0_data_scrapping/results/isw/")
#     for file in files
#         if endswith(file, ".txt")
#             file_path = joinpath(root, file)
#             file_content = read(file_path, String)
#             words_count = length(split(file_content))
#             date = replace(file, "assessment-" => "")
#             date = replace(date, ".txt" => "")
#             date = Date.(date, "yyyy-mm-dd")
#             wordFrequencyToDateFrame = vcat(wordFrequencyToDateFrame, DataFrame(Date=date, Count=words_count))
#         end
#     end
# end

# call do_processing() function in 2_data_prepartion/text_preprocessing.py using pycall
using PyCall

println(PyCall.python)
println(joinpath(@__DIR__, "..", "2_data_preparation"))
pushfirst!(PyVector(pyimport("sys")["path"]), joinpath(@__DIR__, "..", "2_data_preparation"))
text_processing = pyimport("text_preprocessing")


# sort!(wordFrequencyToDateFrame, :Count, rev=true)

# println(describe(wordFrequencyToDateFrame))

# using Plots
# start_date = Date.("2022-02-01", "yyyy-mm-dd")
# end_date = Date.(Dates.now())
# p = plot(wordFrequencyToDateFrame[!, :Date], wordFrequencyToDateFrame[!, :Count], seriestype=:scatter, xlabel="Date", ylabel="Count",
#     xlims=Dates.value.([start_date, end_date]), title="Date vs Total article words count",
#     legend=true, label="count", color="white", background_color="black",
#     grid=false);
# println(p)
# println(word_collection)

In [ ]:
# create dataframe
# 3×7 DataFrame
#  Row │ variable  mean     min                                median   max                                nmissing  eltype   
#      │ Symbol    Nothing  Any                                Nothing  Any                                Int64     DataType 
# ─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
#    1 │ Name               assessment-2022-02-24                       assessment-2023-04-03                     0  String31
#    2 │ Date               2022-02-24                                  2023-04-03                                0  Date
#    3 │ Keywords           {'agent': 0.20139877526758002, '…           {'znpp': 0.26604239982226846, 'c…         0  String

